# Meditation Detail Scrape

In [ ]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import pandas as pd
import string
from datetime import date
from datetime import datetime

In [ ]:
# Insight Timer webpages use javascript, so need selenium and chrome driver.
chrome_driver_path = '../../../../Tech/chrome_driver/chromedriver.exe'

### Create Meditation Detail Batch File
This file serves two purposes. It stores the collected batched results of the meditation list scrape process. It is also used to assign meditations to batches for the meditation detail scrape process.

Records with status 'page not found' or 'meditations not found' are kept in this file as documentation for the results of the meditation list scrape process. These records will not assigned to batches for the meditation detail scrape process, however.

In [ ]:
def create_meditation_detail_batch_file():
    expected_columns = ['teacher_id',
                        'meditation_id',
                        'med_list_batch_id',
                        'med_list_scrape_date',
                        'med_list_scrape_status'
                       ]
    
    #For each batch data file in the teacher_batch_files subdirectory
    batch_files_list = os.listdir('../data/med_list_batch_files/')

    batch_df_list = []

    for batch_file in batch_files_list:
        batch_df = pd.read_csv('../data/med_list_batch_files/' + batch_file, index_col = 0)    

        #Verify the batch has the correct columns in the correct order.
        correct_columns = True

        batch_columns = list(batch_df.columns)
        if len(batch_columns) == len(expected_columns):
            for col_position in range(0,len(batch_columns)):
                if batch_columns[col_position] != expected_columns[col_position]:
                    correct_columns = False
        else:
            correct_columns = False
        
        #Create list of dataframes that will be concatenated into one dataframe.
        if correct_columns:
            batch_df_list = batch_df_list + [batch_df]
        else:
            print('ERROR COLUMNS NOT AS EXPECTED',batch_file)

    global meditations_df
    meditations_df = pd.concat(batch_df_list)
    meditations_df = meditations_df.reset_index(drop = True)
    
    meditations_df.to_csv('../data/med_detail_batch.csv')
    
    print ('Total number of batch files =', len(batch_files_list))
    print ('Batch files consolidated =', len(batch_df_list))
    print ('Batch files with column errors =', len(batch_files_list) - len(batch_df_list))
    print ('-----------------')
    print ('Total number of rows =', meditations_df.shape[0])
    print ('Number of meditations = ', meditations_df.loc[meditations_df.med_list_scrape_status == 'meditation found'].shape[0])
    print ('Number of teachers with no meditations = ', meditations_df.loc[meditations_df.med_list_scrape_status == 'meditations not found'].shape[0])
    print ('Number of teacher pages not found = ', meditations_df.loc[meditations_df.med_list_scrape_status == 'page not found'].shape[0])
    print ('-----------------')
    print ('Number of teachers with meditations =', len(meditations_df.loc[meditations_df.med_list_scrape_status == 'meditation found'].teacher_id.unique()))

In [ ]:
create_meditation_detail_batch_file()

### Function: Assign Batches

Assumptions/Requirements for function:
- The Meditation List Scraping notebook has been run successfully, creating csv batch files in the med_list_batch_files subdirectory.
- Med_detail_batch_df.csv has numeric sequential index values starting with 0.
- The min_batch_size argument passed to the function is an integer value between 0 and the number of rows in teachers_list_df.
- The following data subdirectory for teacher batch files exists: ..\data\med__batch_files

***Note: Once you choose a batch size, you want to avoid reassigning batches with a different batch size. If you do reassign with a different batch size, you can redo it with the original batch size and it will be the same as it was before.***

***Note: Meditations from the same teacher may be assigned to different batches.***

In [ ]:
def assign_batches(batch_size):
    
    global med_detail_batch_df
    med_detail_batch_df = pd.read_csv('../data/med_detail_batch.csv', index_col=0)
    
    #Take subset of columns to drop any prior med detail batch assignment.
    med_detail_batch_df = med_detail_batch_df[['teacher_id',
                                               'meditation_id',
                                               'med_list_batch_id',
                                               'med_list_scrape_date',
                                               'med_list_scrape_status']]
    
    num_rows = med_detail_batch_df.loc[med_detail_batch_df.med_list_scrape_status == 'meditation found'].shape[0]    
    
    #Verify that the index values are as expected (0, 1, 2, etc.)
    index_list = med_detail_batch_df.index.to_list()
    for x in list(range(0,num_rows)):
        assert index_list[x] == x

    skipped_rows = 0
    
    for index, row in med_detail_batch_df.iterrows():
        if row.med_list_scrape_status == 'meditation found':
            med_detail_batch_df.loc[index, 'med_detail_batch_id'] = (((index - skipped_rows) - ((index - skipped_rows) % batch_size)) / batch_size) + 1
        else:
            skipped_rows += 1
            med_detail_batch_df.loc[index, 'med_detail_batch_id'] = -999

    med_detail_batch_df.med_detail_batch_id = med_detail_batch_df.med_detail_batch_id.astype('int')
                                
    #Save batch results to data file
    med_detail_batch_df.to_csv('../data/med_detail_batch.csv')
    
    print(num_rows + skipped_rows,'total rows')
    print(num_rows, 'batched rows')
    print(skipped_rows, 'skipped rows')
    print(batch_size,'rows per batch')
    num_batches = ((num_rows - (num_rows % batch_size)) / batch_size) + min(1,num_rows % batch_size)
    print(int(num_batches),'batches')
    

In [ ]:
assign_batches(100)

In [ ]:
med_detail_batch_df.head()

### Function: Scrape Teachers One Batch

Assumptions/Requirements for function:
- The assign_batches function has been run to create the med_detail_batch.csv file.
- Dataframe med_detail_batch_df has been created by reading in med_detail_batch.csv file.
- med_detail_batch_df has numeric sequential index values starting with 0.
- The scrape_batch_id argument passed to the function is an integer that matches an index value for a row in med_detail_batch_df.

In [ ]:
def scrape_meditations_one_batch(scrape_batch_id):
    meditations_batch = med_detail_batch.copy().loc[med_detail_batch.med_detail_batch_id == scrape_batch_id]
    print(meditations_batch.shape[0],'rows in batch')
    
    #Batch Start
    batch_start_time = datetime.now()
    batch_row = 0
    prior_teacher_id = '----------'
    teacher_meditation_row = 0

    for index, row in meditations_batch.iterrows():
        
        batch_row += 1
        if row.teacher_id == prior_teacher_id:
            teacher_meditation_row += 1
        else:
            teacher_meditation_row = 1
            prior_teacher_id = row.teacher_id
            
        print('Batch row',batch_row,'of',
              meditations_batch.shape[0],
              ': teacher_id =', 
              row.teacher_id,
              'meditation #',
              teacher_meditation_row)

        #Record scrape date (Rows will have scrape_date even if page not found.)
        meditations_batch.loc[index, 'med_detail_scrape_date'] = datetime.now()

        #All meditation_id values start with '/guided-meditations/'
        #Would have removed '/guieded-meditations/' from meditation_id, but during testing some meditation 
        #urls had a different format, and the following code worked for both formats.
        meditation_url = 'https://insighttimer.com' + row.meditation_id

        driver = webdriver.Chrome(executable_path=chrome_driver_path)
        driver.get(meditation_url)

        #Wait for page to fully load
        driver.implicitly_wait(2)  #Two seconds is usually but not always long enough.
        
        #Make soup
        soup = BS(driver.page_source)
    
        h1_page_not_found_tag = soup.find('h1', attrs = {'class':'text-lg font-ProxiBold mb-6 leading-tight'})
        if h1_page_not_found_tag is None:
            page_found = True
        elif h1_page_not_found_tag.text == "The page you were looking for doesn't exist.":
            page_found = False
        else:
            page_found = True
        
        if page_found == False:
            meditations_batch.loc[index, 'med_detail_scrape_status'] = 'page not found'
            print('PAGE NOT FOUND for teacher_id =',
                  row.teacher_id,
                  'meditation #',
                  teacher_meditation_row,
                  'meditation_id',
                  row.meditation_id)
            driver.close() #Close driver (closes browser window)

        else:
            #Get meditation title. If not found, wait longer, remake soup, and look again.
            h1_title_tag = soup.find('h1', attrs = {'class':'font-ProxiBold sm:text-3xl2 text-2xl2 w-full leading-tight mb-2'})
        
            if h1_title_tag is None:
                driver.implicitly_wait(5)
                soup = BS(driver.page_source)
                h1_title_tag = soup.find('h1', attrs = {'class':'font-ProxiBold sm:text-3xl2 text-2xl2 w-full leading-tight mb-2'})

            driver.close() #Close driver (closes browser window)
            
            if h1_title_tag is None:
                meditations_batch.loc[index, 'med_detail_scrape_status'] = 'title not found'
                print('MEDITATION TITLE NOT FOUND for teacher_id =',
                      row.teacher_id, 
                      'meditation #',
                      teacher_meditation_row,
                      'meditation_id',
                      row.meditation_id)
            else:
                meditations_batch.loc[index, 'med_detail_scrape_status'] = 'title found'
                meditations_batch.loc[index, 'meditation_title'] = h1_title_tag.text
                               
#Should this be indented?
            #Get length
            h2_length_tag = soup.find('h2', attrs = {'class':'font-ProxiRegular text-5xl text-white tracking-wider relative z-10'})
            if h2_length_tag is not None:
                meditations_batch.loc[index, 'length'] = h2_length_tag.text
                
            #Get plays
                
            #Get rating
            p_rating_tag = soup.find('p', attrs = {'class':'font-ProxiSemibold text-x1 flex items-center mx-auto'})
            if p_rating_tag is not None:
                meditations_batch.loc[index, 'rating'] = p_rating_tag.text
                
            #Get number of reviews
            
            #Get upload date
            
            #Get type
            
            #Get activity
            
            #Get suitable for
            
            #Get topics
            
            #Get description
            p_description_tag = soup.find('p', attrs = {'class':'text-base sm:text-lg tracking-wide leading-relaxed font-ProxiRegular'})
            if p_description_tag is not None:
                meditations_batch.loc[index, 'description'] = p_description_tag.text
                
            #Get image url
            div_image_tag = soup.find('div', attrs = {'class':'jss1444 md:rounded-lg absolute top-0 left-0 bg-no-repeat bg-cover overflow-hidden'})
            if div_image_tag is not None:
                meditations_batch.loc[index, 'image'] = div_image_tag.get('style', default = 'No url') 
                
    #Save batch results to data file
    batch_id_string = str(scrape_batch_id).zfill(5)  
    #zfill adds leading zeros which allows filenames to be sorted correctly alphabetically.
    batch_filename = '../data/med_detail_batch_files/med_detail_batch_' + str(batch_id_string) + '.csv'
    meditations_batch.to_csv(batch_filename)

    #Batch End
    batch_end_time = datetime.now()

    #Print Runtime 
    batch_runtime = batch_end_time - batch_start_time
    hours, remainder = divmod(batch_runtime.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    print('Batch_id',scrape_batch_id,'completed')
    print('Batch runtime:')

    if hours > 0:
        print(hours,'hours')
    if minutes > 0:
        print(minutes,'minutes')
    print(seconds,'seconds')

Meditation Columns
- title
- length
- plays
- rating
- num_reviews
- upload_date
- type
- activity
- suitable_for
- topics
- description
- image

### Example use of scrape_teachers_one_batch function

In [ ]:
med_detail_batch = pd.read_csv('../data/med_detail_batch.csv', index_col=0)

#Verify that the index values are as expected (0, 1, 2, etc.)
index_list = med_detail_batch.index.to_list()
for x in list(range(0,med_detail_batch.shape[0])):
    assert index_list[x] == x

In [ ]:
scrape_batch_id = 1551
scrape_meditations_one_batch(scrape_batch_id)

In [ ]:
fresh_batch = pd.read_csv('../data/med_detail_batch_files/med_detail_batch_00085.csv', index_col=0)
fresh_batch.head()

# OLD DRAFT

In [ ]:
#meditation_url = ''
meditation_url = 'https://insighttimer.com/malhuxter/guided-meditations/the-meditation-hindrance-of-paralysing-doubt'
#meditation_url = ''

In [ ]:
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get(meditation_url)

#Wait for page to fully load
driver.implicitly_wait(2)

#Make soup and close driver
soup = BS(driver.page_source)
driver.close()

In [ ]:
#Get meditation name -- Will check for matches with name scraped for list
h1_meditation_name_tag = soup.find('h1', attrs = {'class':'font-ProxiBold sm:text-3xl2 text-2xl2 w-full leading-tight mb-2'})
meditation_name = h1_meditation_name_tag.text
meditation_name

In [ ]:
#Get teacher
h3_teacher_tag = soup.find('h1', attrs = {'class':'font-ProxiBold text-2xl4 w-full hover:opacity-75 break-words pre-wrap'})
meditation_name = h1_meditation_name_tag.text
meditation_name

<h3 class="font-ProxiBold text-2xl4 w-full hover:opacity-75 break-words pre-wrap">Malcolm Huxter</h3>

<a href="/malhuxter" aria-current="page" class="active"> Malcolm Huxter </a>

In [ ]:
#get type

<svg width="10" height="14" viewBox="0 0 10 14" class="h-5 w-5 mr-1 "><path d="M9.593 6v1a4.34 4.34 0 0 1-1.152 3.004c-.769.851-1.718 1.34-2.848 1.465V12.5h2a.48.48 0 0 1 .352.148.48.48 0 0 1 .148.352.48.48 0 0 1-.148.352.48.48 0 0 1-.352.148h-5a.48.48 0 0 1-.352-.148.48.48 0 0 1-.148-.352.48.48 0 0 1 .148-.352.48.48 0 0 1 .352-.148h2v-1.031c-1.13-.125-2.08-.614-2.848-1.465A4.34 4.34 0 0 1 .593 7V6a.48.48 0 0 1 .148-.352.48.48 0 0 1 .352-.148.48.48 0 0 1 .352.148.48.48 0 0 1 .148.352v1A3.37 3.37 0 0 0 2.62 9.473 3.37 3.37 0 0 0 5.093 10.5a3.37 3.37 0 0 0 2.473-1.027A3.37 3.37 0 0 0 8.593 7V6a.48.48 0 0 1 .148-.352.48.48 0 0 1 .352-.148.48.48 0 0 1 .352.148.48.48 0 0 1 .148.352zm-2-3v4c0 .688-.245 1.276-.734 1.766-.49.49-1.078.734-1.766.734a2.407 2.407 0 0 1-1.766-.734A2.407 2.407 0 0 1 2.593 7V3c0-.688.245-1.276.734-1.766A2.407 2.407 0 0 1 5.093.5c.688 0 1.276.245 1.766.734.49.49.734 1.079.734 1.766z"></path></svg>


In [ ]:
#get activity

<div class="font-ProxiSemibold text-xl">Meditation</div>

In [ ]:
#Get suitable for

<div class="font-ProxiSemibold text-xl">Experienced</div>

In [ ]:
#Get length

<h2 class="font-ProxiRegular text-5xl text-white tracking-wider relative z-10">07:30</h2>

In [ ]:
#get tags

<div class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-1"><div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/lectures"><span class="MuiButton-label"><span>Lectures</span></span><span class="MuiTouchRipple-root"></span></a></div><div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/spiritual"><span class="MuiButton-label"><span>Spiritual</span></span><span class="MuiTouchRipple-root"></span></a></div><div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/freedomfromsuffering"><span class="MuiButton-label"><span>Freedom from Suffering</span></span><span class="MuiTouchRipple-root"></span></a></div><div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/buddhism"><span class="MuiButton-label"><span>Buddhism</span></span><span class="MuiTouchRipple-root"></span></a></div><div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/contemporarybuddhism"><span class="MuiButton-label"><span>Contemporary Buddhism</span></span><span class="MuiTouchRipple-root"></span></a></div><div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/parentspirituality"><span class="MuiButton-label"><span>Spirituality</span></span><span class="MuiTouchRipple-root"></span></a></div></div>

In [ ]:
#Get rating
p_rating_tag = soup.find('h1', attrs = {'class':'font-ProxiSemibold text-x1 flex items-center mx-auto'})
rating = p_rating_tag.text
rating

In [ ]:
#Get number of reviews
<div class="w-full">
    <h2 class="css-1dvmp89">Recent Reviews</h2>
    <div class="css-gm93j8">
        <svg class="MuiSvgIcon-root css-14icxgp" focusable="false" viewBox="0 0 24 24" aria-hidden="true" role="presentation"><path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path><path fill="none" d="M0 0h24v24H0z"></path></svg>
        <span>4.78</span>
        <div class="css-1651lgy"></div>
        <span>39397</span></div>

In [ ]:
#get plays

<div class="font-ProxiSemibold text-xl">68</div>

In [ ]:
#image for meditation?

#from meditation page
<div class="jss1444 md:rounded-lg absolute top-0 left-0 bg-no-repeat bg-cover overflow-hidden" style="background-image: url(&quot;https://libraryitems.insighttimer.com/d8e8d4v1l6u6a8w8j7e2f0r6s5f6b7t8j9d6t3j5/pictures/tiny_rectangle_xlarge.jpeg&quot;); background-position-y: 25%;"></div>

#from teacher tracks page
<img src="https://libraryitems.insighttimer.com/d8e8d4v1l6u6a8w8j7e2f0r6s5f6b7t8j9d6t3j5/pictures/tiny_rectangle_medium.jpeg" class="chakra-image css-1hfz4mu">

In [ ]:
#Get description
<p class="text-base sm:text-lg tracking-wide leading-relaxed font-ProxiRegular">Welcome to this grounding guided meditation for calming the monkey mind and coming into deeper focus, calm, and concentration. This meditation will support you in:

Quieting overactive thoughts and mental distractions
Increased focus and concentration
Receiving energies of peace and calm 
Returning to presence 

I hope you enjoy xx
</p>

In [ ]:
#Get uploadDate

In [ ]:
<div class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-4">
    #1 Title
    <div class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12">
        <h1 class="font-ProxiBold sm:text-3xl2 text-2xl2 w-full leading-tight mb-2">Re-Centering In Times Of Uncertainty</h1>
        <p class="font-ProxiSemibold text-grey_ text-base">by <a href="/alex_elle" aria-current="page" class="active"> Alex Elle </a></p></div>

    #2 Rating
    <div class="MuiGrid-root flex MuiGrid-item">
        <svg width="22px" height="43px" viewBox="0 0 22 43"><title>rating.1a6a70b7</title><g id="Page-1" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><g id="Letter" transform="translate(-141.000000, -437.000000)" fill="#CCCCCC"><g id="rating.1a6a70b7" transform="translate(141.000000, 437.000000)"><path d="M20.919,41.292 C20.2834517,41.1677393 19.6634226,40.9742527 19.07,40.715 C18.144,35.548 13.893,32.437 13.893,32.437 C12.916,36.942 16.413,39.721 18.191,40.815 C16.549,40.047 12.801,38.783 6.7,39.359 C6.7,39.359 12.473,45.019 18.807,41.556 C19.537,41.792 20.304,41.966 20.763,42.062 C20.870434,42.0846885 20.9824669,42.061868 21.072466,41.9989633 C21.162465,41.9360587 21.2223917,41.8386886 21.238,41.73 C21.2648149,41.5221254 21.1241889,41.3296436 20.918,41.292" id="Path"></path><path d="M11.31,37.36 C11.873,37.799 12.487,38.196 12.857,38.425 C13.04,38.538 13.28,38.474 13.381,38.285 C13.4715364,38.1129288 13.411291,37.9000911 13.244,37.801 C12.7285659,37.4903971 12.2489948,37.1238438 11.814,36.708 C12.684,31.893 9.974,27.819 9.974,27.819 C7.655,31.451 9.812,34.994 11.012,36.517 C9.825,35.322 6.956,33.025 1.43,31.59 C1.43,31.59 4.642,38.379 11.31,37.36" id="Path"></path><path d="M6.682,31.673 C7.022,32.259 7.424,32.825 7.67,33.158 C7.72835711,33.2364201 7.81646776,33.2873071 7.91355701,33.2986625 C8.01064627,33.310018 8.10812158,33.280837 8.183,33.218 C8.32550185,33.0988208 8.34886489,32.8885533 8.236,32.741 C7.89075009,32.2853033 7.5972778,31.7926052 7.361,31.272 C9.857,27.312 8.919,22.756 8.919,22.756 C5.563,25.16 6.202,29.053 6.719,30.822 C6.096,29.348 4.379,26.309 1.77635684e-15,23.102 C1.77635684e-15,23.102 0.41,30.239 6.682,31.673" id="Path"></path><path d="M4.497,24.727 C4.527,25.401 4.62,26.087 4.684,26.494 C4.716,26.694 4.912,26.827 5.112,26.781 C5.29382611,26.740498 5.41206812,26.5646734 5.381,26.381 C5.28630151,25.8189465 5.25477213,25.2480636 5.287,24.679 C9.345,22.298 10.631,17.825 10.631,17.825 C6.533,18.435 5.291,22.182 4.927,23.987 C5.059,22.396 4.948,18.917 2.551,14.077 C2.551,14.077 -0.404,20.602 4.497,24.727" id="Path"></path><path d="M5.482,19.188 C5.47586854,19.273601 5.5057299,19.3578784 5.56438802,19.420523 C5.62304614,19.4831676 5.70518116,19.5184979 5.791,19.5180052 C5.95469247,19.518341 6.09043953,19.3913518 6.101,19.228 C6.143,18.579 6.275,18.048 6.364,17.754 C10.317,16.514 12.318,12.94 12.318,12.94 C8.688,12.655 6.87,15.617 6.195,17.091 C6.629,15.754 7.235,12.755 6.157,8.142 C6.157,8.142 2.315,13.144 5.678,17.639 C5.568,18.222 5.51,18.827 5.482,19.189" id="Path"></path><path d="M7.927,12.565 C7.90886607,12.6453161 7.92512379,12.7295561 7.97184307,12.797356 C8.01856234,12.865156 8.09149118,12.9103453 8.173,12.922 C8.32858826,12.9450823 8.47545703,12.8436729 8.509,12.69 C8.61103274,12.2192922 8.76532259,11.7614539 8.969,11.325 C12.905,10.7 15.318,7.582 15.318,7.582 C11.908,6.802 9.756,9.365 8.904,10.672 C9.507,9.462 10.513,6.695 10.148,2.16 C10.148,2.16 5.78,6.377 8.335,11.12 C8.147,11.66 8.005,12.226 7.927,12.565" id="Path"></path><path d="M17.386,0.142 C17.386,0.142 11.178,1.412 10.953,7.03 C10.953,7.03 14.355,5.758 17.386,0.142" id="Path"></path></g></g></g></svg>
        <div class="flex flex-col justify-center items-start bg-center bg-contain bg-no-repeat">
            <div class="font-ProxiRegular text-black50 text-sm leading-relaxed mx-auto">Rated</div>
            <p class="font-ProxiSemibold text-xl flex items-center mx-auto">4.8 
                <svg class="MuiSvgIcon-root jss638 text-black75" focusable="false" viewBox="0 0 24 24" aria-hidden="true" role="presentation"><path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path><path fill="none" d="M0 0h24v24H0z"></path></svg></p></div>
        <svg width="22px" height="43px" viewBox="0 0 22 43"><title>Group</title><g id="Page-1" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><g id="Letter" transform="translate(-207.000000, -437.000000)" fill="#CCCCCC"><g id="Group" transform="translate(207.000000, 437.000000)"><path d="M0.32,41.292 C0.955889207,41.1678099 1.57625736,40.9743221 2.17,40.715 C3.096,35.548 7.347,32.437 7.347,32.437 C8.323,36.942 4.827,39.721 3.048,40.815 C4.691,40.047 8.438,38.783 14.54,39.359 C14.54,39.359 8.766,45.019 2.433,41.556 C1.703,41.792 0.935,41.966 0.477,42.062 C0.369272345,42.0853136 0.25670128,42.0627918 0.166235362,41.999826 C0.0757694454,41.9368602 0.0155525172,41.8391187 0,41.73 C-0.0268148663,41.5221254 0.113811111,41.3296436 0.32,41.292" id="Path"></path><path d="M9.929,37.36 C9.367,37.799 8.753,38.196 8.382,38.425 C8.29481814,38.4788626 8.18927797,38.4943745 8.09028478,38.4678754 C7.9912916,38.4413763 7.90761467,38.3752135 7.859,38.285 C7.76846363,38.1129288 7.82870901,37.9000911 7.996,37.801 C8.51143409,37.4903971 8.99100521,37.1238438 9.426,36.708 C8.556,31.893 11.265,27.819 11.265,27.819 C13.585,31.451 11.428,34.994 10.228,36.517 C11.415,35.322 14.284,33.025 19.81,31.59 C19.81,31.59 16.598,38.379 9.929,37.36" id="Path"></path><path d="M14.558,31.673 C14.218,32.259 13.816,32.825 13.57,33.158 C13.5116355,33.2367283 13.4233213,33.2878266 13.3259793,33.2991895 C13.2286374,33.3105523 13.1309284,33.2811688 13.056,33.218 C12.9138163,33.0985325 12.8908976,32.8882974 13.004,32.741 C13.453,32.148 13.734,31.592 13.878,31.272 C11.383,27.312 12.321,22.756 12.321,22.756 C15.677,25.16 15.037,29.053 14.52,30.822 C15.143,29.348 16.86,26.309 21.24,23.102 C21.24,23.102 20.829,30.239 14.558,31.673" id="Path"></path><path d="M16.742,24.727 C16.712,25.401 16.619,26.087 16.555,26.494 C16.5392331,26.5901446 16.4844303,26.6754975 16.4035691,26.7298468 C16.322708,26.7841961 16.2229796,26.8027084 16.128,26.781 L16.127,26.781 C15.9455784,26.7400506 15.8278666,26.5643613 15.859,26.381 C15.982,25.651 15.974,25.029 15.953,24.679 C11.894,22.298 10.608,17.825 10.608,17.825 C14.707,18.435 15.949,22.182 16.313,23.987 C16.181,22.396 16.292,18.917 18.689,14.077 C18.689,14.077 21.644,20.602 16.742,24.727" id="Path"></path><path d="M15.757,19.188 C15.7631178,19.2734268 15.7333893,19.3575434 15.6749518,19.420155 C15.6165143,19.4827667 15.5346453,19.5182188 15.449,19.518001 C15.2853075,19.518341 15.1495605,19.3913518 15.139,19.228 C15.1065314,18.7284017 15.018317,18.2339985 14.876,17.754 C10.923,16.514 8.922,12.94 8.922,12.94 C12.552,12.655 14.37,15.617 15.045,17.091 C14.611,15.754 14.005,12.755 15.082,8.142 C15.082,8.142 18.925,13.144 15.562,17.639 C15.672,18.222 15.73,18.827 15.757,19.189" id="Path"></path><path d="M13.313,12.565 C13.3311339,12.6453161 13.3148762,12.7295561 13.2681569,12.797356 C13.2214377,12.865156 13.1485088,12.9103453 13.067,12.922 C12.9110584,12.9456569 12.763559,12.8441143 12.73,12.69 C12.6279673,12.2192922 12.4736774,11.7614539 12.27,11.325 C8.335,10.7 5.922,7.582 5.922,7.582 C9.332,6.802 11.483,9.365 12.336,10.672 C11.732,9.462 10.727,6.695 11.091,2.16 C11.091,2.16 15.46,6.377 12.905,11.12 C13.093,11.66 13.235,12.226 13.313,12.565" id="Path"></path><path d="M3.853,0.142 C3.853,0.142 10.061,1.412 10.286,7.03 C10.286,7.03 6.884,5.758 3.853,0.142" id="Path"></path></g></g></g></svg></div>

    #3 Type
    <div class="MuiGrid-root MuiGrid-item">
        <div class="flex flex-col justify-center items-start">
            <div class="font-ProxiRegular text-black50 text-sm leading-relaxed">Type</div>
            <div class="font-ProxiSemibold text-xl capitalize flex flex-row items-center">
                <svg width="24" height="23" viewBox="0 0 24 23" class="h-5 w-5 mr-1 ">
                    <g fill="#181818" fill-rule="evenodd">
                        <path d="M10.87 10.574c0-.495-.405-.9-.9-.9a.9.9 0 0 0-.9.9v3.429a.6.6 0 1 1-1.2 0v-3.429a2.1 2.1 0 0 1 1.233-1.908l1.552-5.125A3.066 3.066 0 0 0 9.06 0L3.44 9.702 2.632 15.8.483 16.964a.5.5 0 0 0-.177.718l2.968 4.429a.5.5 0 0 0 .686.142l4.705-3.037a4.8 4.8 0 0 0 2.205-4.038v-4.603zM20.7 9.702L15.08 0a3.067 3.067 0 0 0-1.596 3.54l1.551 5.127a2.098 2.098 0 0 1 1.234 1.908v3.428a.6.6 0 0 1-1.2 0v-3.428a.9.9 0 0 0-1.799 0v4.602c0 1.634.83 3.155 2.204 4.04l4.704 3.036a.5.5 0 0 0 .687-.142l2.968-4.43a.5.5 0 0 0-.178-.717L21.508 15.8l-.809-6.099z"></path>
                    </g>
                </svg>guided
            </div>
        </div>
    </div>

    #4 Activity
    <div class="MuiGrid-root MuiGrid-item">
        <div class="flex flex-col justify-center items-start">
            <div class="font-ProxiRegular text-black50 text-sm leading-relaxed">Activity</div>
            <div class="font-ProxiSemibold text-xl">Meditation</div></div></div>

    #5 Suitable for
    <div class="MuiGrid-root MuiGrid-item">
        <div class="flex flex-col justify-center items-start">
            <div class="font-ProxiRegular text-black50 text-sm leading-relaxed">Suitable for</div>
            <div class="font-ProxiSemibold text-xl">Everyone</div></div></div>

    #6 Plays
    <div class="MuiGrid-root MuiGrid-item">
        <div class="flex flex-col justify-center items-start">
            <div class="font-ProxiRegular text-black50 text-sm leading-relaxed">Plays</div>
            <div class="font-ProxiSemibold text-xl">365k</div></div></div>

    #7 Description
    <div class="MuiGrid-root MuiGrid-item MuiGrid-grid-md-12">
        <p class="text-base sm:text-lg tracking-wide leading-relaxed font-ProxiRegular">It's no secret that collectively the world is going through a scary and uncertain time. This meditation was created for re-centering in times of fear, chaos, and uncertainty. We cannot always control what is going on outside of us, but we can always come back to our breath.</p></div>

    #8 Topics  6 div class 'MuiGrid-root MuiGrid-item' each has a topic
    <div class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12">
        <div class="MuiGrid-root MuiGrid-container MuiGrid-spacing-xs-1">
            <div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/anxiety"><span class="MuiButton-label"><span>Anxiety</span></span><span class="MuiTouchRipple-root"></span></a></div>
            <div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/stressanxiety"><span class="MuiButton-label"><span>Stress &amp; Anxiety</span></span><span class="MuiTouchRipple-root"></span></a></div>
            <div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/energybased"><span class="MuiButton-label"><span>Energy-Based</span></span><span class="MuiTouchRipple-root"></span></a></div>
            <div class="MuiGrid-root MuiGrid-item"><a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/concentration"><span class="MuiButton-label"><span>Concentration</span></span><span class="MuiTouchRipple-root"></span></a></div>
            <div class="MuiGrid-root MuiGrid-item">
                <a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/parentspirituality">
                    <span class="MuiButton-label"><span>Spirituality</span></span>
                    <span class="MuiTouchRipple-root"></span></a></div>
            <div class="MuiGrid-root MuiGrid-item">
                <a class="MuiButtonBase-root MuiButton-root MuiButton-text outline-none font-ProxiSemibold text-lg tracking-wide text-xdark_ bg-xlightgrey_ py-2 px-6 hover:bg-black14" tabindex="0" role="button" aria-disabled="false" href="/meditation-topics/breathingmeditation">
                    <span class="MuiButton-label"><span>Breathing Meditation</span></span>
                    <span class="MuiTouchRipple-root"></span></a></div></div></div>
    #9
    <div class="MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12"></div></div>